# Báo cáo đồ án môn "Khoa học dữ liệu ứng dụng"

Nhóm 2:

1.   1712892 - Nguyễn Thị Tuyết - (Đang bệnh)
2.   1712898 - Trần Việt Văn - tvvan99
3.   1712685 - Trần Huy Phượng - nomiku1999

Link thùng chứa Github của nhóm: https://github.com/nomiku1999/KTDLUD


## Mô tả bài toán

* **Tên cuộc thi**: March Machine Learning Mania 2021 - NCAAM
Predict the 2021 NCAAM Basketball Tournament

* **Mô tả**: Dự đoán kết quả của giải thi đấu bóng rổ NCAA 2021 dành cho nam dựa trên dữ liệu lịch sử trận đấu được lưu trữ trước đó
* **Input**: Dữ liệu được cung cấp bao gồm lịch sử các trận đấu của giải
* **Output**: Model dự đoán kết quả của các trận đấu, tỉ lệ thắng của đội có id đứng trước so với đội có id đứng sau

Link to competition
https://www.kaggle.com/competitions/ncaam-march-mania-2021/overview/description

## Giải quyết bài toán

###1.Ý tưởng

Sử dụng các thuật toán machine learning để dự đoán kết quả trận đấu dựa trên dữ liệu các trận đấu trong lịch sử, train 1 vài model thông dụng để chọn được model với metric tốt nhất cho dataset

Sử dụng dữ liệu từ 3 csv file:
* MRegularSeasonCompactResults: dữ liệu lịch sử trận đấu, gồm đội thắng, đội thua, điểm số
* MRegularSeasonDetailedResults: dữ liệu lịch sử chi tiết bao gồm thêm 1 số field chứa thông tin thêm của trận đấu
* MMasseyOrdinals: dữ liệu chứa ranking của các đội

Submission file:
* MSampleSubmission chứa 2 field, field đầu tiên là id của trận đấu, field thứ 2 chứa xác suất team có id nhỏ hơn đứng trước thắng trận đấu (field này phải predict)

Ex: 

    2015_1107_1110,0.5
    2015_1107_1112,0.5
    2015_1107_1113,0.5

Số đầu tiên biểu thị cho năm, số thứ 2 là id của đội đầu tiên, số thứ 3 là id của đội thứ hai, số cuối cùng (sau dấy phẩy) biểu thị xác suất đội đầu tiên thắng đội thứ 2

###2. Solution

####a/ Thông tin chung

Nhóm sẽ sử dụng solution rank 9 tại private leaderboard

Link to solution: https://www.kaggle.com/c/ncaam-march-mania-2021/discussion/230991

Data input: Dữ liệu cũ đến trước năm 2015

Data test: Dữ liệu từ 2015 trở đi



####b/ Tóm tắt các bước thực hiện



1.   Cleaning and extracting data from source
2.   Build multiple model for predicting
3.   Calculating metric for algorithms, choose the best model for predicting match result



####c/ Lí do lựa chọn

Lí do lựa chọn:
* Sử dụng data dạng table, dễ thao tác, tối ưu hóa phần cứng.
* Solution sử dụng cùng lúc nhiều model để predict, các model dễ hiểu và tiếp cận với nhóm.

###3. Cài đặt

## Nhìn lại quá trình làm đồ án

## Tài liệu tham khảo